In [1]:
import cv2

# Kamerayı başlat
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Webcam", frame)

    # 'q' tuşuna basınca çık
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2025-03-17 15:47:54.065 Python[97065:5807676] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-17 15:47:54.065 Python[97065:5807676] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [2]:
import cv2
import mediapipe as mp

# Yüz tespiti için MediaPipe modelini yükle
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Kamerayı başlat
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV'nin BGR formatını RGB'ye çevir
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # MediaPipe ile yüz analizi yap
    results = face_mesh.process(rgb_frame)

    # Eğer yüz algılanırsa çizim yap
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                # Pixel koordinatlarını al
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                
                # Yüz noktalarını çiz
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    cv2.imshow("Face Mesh", frame)

    # 'q' tuşuna basınca çık
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1742215748.520855 5807676 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742215748.534872 5809655 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742215748.539512 5809655 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742215749.887300 5809657 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Emoji eklemek için Unicode karakterleri
EMOJI_MAP = {
    "neutral": "😐",
    "happy": "😃",
    "sad": "😢",
    "surprised": "😲"
}

# MediaPipe yüz tespiti başlat
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Kamerayı aç
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV'den RGB'ye çevir
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Varsayılan olarak neutral emoji
    current_emoji = EMOJI_MAP["neutral"]

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            
            # Ağız açıklığını hesapla (üst dudak ve alt dudak mesafesi)
            top_lip = np.array([landmarks[13].x, landmarks[13].y])
            bottom_lip = np.array([landmarks[14].x, landmarks[14].y])
            mouth_distance = np.linalg.norm(top_lip - bottom_lip)

            # Kaş yüksekliğini hesapla (kaşlar ile gözler arası mesafe)
            left_brow = np.array([landmarks[55].x, landmarks[55].y])
            left_eye = np.array([landmarks[33].x, landmarks[33].y])
            brow_distance = np.linalg.norm(left_brow - left_eye)

            # Mutluluk tespiti (ağız çok açık => mutlu)
            if mouth_distance > 0.05:
                current_emoji = EMOJI_MAP["happy"]
            # Üzüntü tespiti (kaşlar çok aşağıda)
            elif brow_distance < 0.02:
                current_emoji = EMOJI_MAP["sad"]
            # Şaşkınlık tespiti (ağız çok fazla açık)
            elif mouth_distance > 0.08:
                current_emoji = EMOJI_MAP["surprised"]

    # Emoji'yi ekrana bas
    cv2.putText(frame, current_emoji, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    cv2.imshow("Emoji Face", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
